In [68]:
# Import libraries
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [127]:
stores = pd.read_csv("C:/Users/daisu/OneDrive/Desktop/GCI/drive-download-20201224T003704Z-001/stores.csv", )
genres = pd.read_csv("C:/Users/daisu/OneDrive/Desktop/GCI/drive-download-20201224T003704Z-001/genres.csv", )
goods = pd.read_csv("C:/Users/daisu/OneDrive/Desktop/GCI/drive-download-20201224T003704Z-001/goods.csv", )
train_df = pd.read_csv("C:/Users/daisu/OneDrive/Desktop/GCI/drive-download-20201224T003704Z-001/train.csv")
test_df  =pd.read_csv("C:/Users/daisu/OneDrive/Desktop/GCI/drive-download-20201224T003704Z-001/test.csv" )

In [128]:
# Add goods_genre_id
train_df = pd.merge(train_df, goods[['goods_id', 'goods_genre_id']], on='goods_id')
test_df = pd.merge(test_df, goods[['goods_id', 'goods_genre_id']], on='goods_id')


# Add shifted features 
train_df['units_sold_month'] = train_df.groupby(['num_month', 'store_id',  'goods_id']).units_sold_day.transform(np.sum)

# Delete duplicated rows
train_df = train_df.drop_duplicates(subset=['num_month', 'store_id', 'goods_id']).reset_index(drop=True)

# Delete unused features' values
train_df.drop(['yy_mm_dd', 'units_sold_day'], axis=1, inplace=True)

# Add 0 on test data's num_month
test_df['num_month'] = 0

In [129]:
all_df = pd.concat([train_df, test_df])
# Generate shifted features
n = 5
for i in range(n):
    all_df[f'units_sold_month_{i+1}'] = all_df.groupby(['store_id', 'goods_id']).units_sold_month.shift(i+1)

In [130]:
# Generate statistic of features
def stats_features(df, columns1, columns2):
    for column1 in columns1:
        for column2 in columns2:
            df[f'{column1}_{column2}_mean'] = df[column1].map(train_df.groupby(column1)[column2].mean())
            df[f'{column1}_{column2}_median'] = df[column1].map(train_df.groupby(column1)[column2].median())
            df[f'{column1}_{column2}_std'] = df[column1].map(train_df.groupby(column1)[column2].std())
    
    return df

In [132]:
# Gnerate statistic of features
columns1 = ['goods_genre_id', 'store_id', 'goods_id']
columns2 = ['units_sold_month', 'price']

all_df = stats_features(all_df, columns1, columns2)

In [134]:
# Split into train and test
train_df = all_df[:1719470]
test_df = all_df[1719470:]

In [135]:
# Prepare dataset for training
X_df = train_df[train_df['num_month'] > -12].reset_index(drop=True).drop(['price', 'units_sold_month', 'index'], axis=1)
y_df = train_df[train_df['num_month'] > -12].reset_index(drop=True).units_sold_month
X_test_df = test_df.sort_values('index').reset_index(drop=True).drop(['price', 'units_sold_month', 'index'], axis=1)

X = X_df.values
y = y_df.values
X_test = X_test_df #.values

In [138]:
# Hyparaparameters from optuna
params = {
          'objective': 'regression',
          'metric': 'rmse',
          'lambda_l1': 1.731879036922e-06, 
          'lambda_l2': 0.008249579034746, 
          'max_depth': 3, 
          'num_leaves': 10, 
          'feature_fraction': 0.99996999864248, 
          'bagging_fraction': 0.63648252566587, 
          'bagging_freq': 5, 
          'min_child_samples': 7,
          'num_boost_round': 273,
          'random_state': 42}

In [139]:
# Predict 
import lightgbm as lgb
lgb_train = lgb.Dataset(X, y)
lgbm = lgb.train(params, lgb_train)
lgb_pred = lgbm.predict(X_test)
lgb_pred = lgb_pred.astype(int)
lgb_pred = np.where(lgb_pred < 0, 0, lgb_pred)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009907 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3036
[LightGBM] [Info] Number of data points in the train set: 467420, number of used features: 26
[LightGBM] [Info] Start training from score 1.990490
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

In [140]:
# Submission file
submission  =pd.read_csv("C:/Users/daisu/OneDrive/Desktop/GCI/drive-download-20201224T003704Z-001/sample_submission.csv", index_col=0 )
submission["units_sold_month"] = lgb_pred
submission

,units_sold_month
index,
0,1
1,0
2,1
3,1
4,0
...,...
214195,0
214196,1
214197,1


In [141]:
# Download the submission file
submission.to_csv('C:/Users/daisu/OneDrive/Desktop/GCI/drive-download-20201224T003704Z-001/late_lgb_cliped.csv',index=False)